input_test, lowest location from range 46

NOT WORKING

In [1]:
import numpy as np
import re

In [2]:
#with open("input_test.txt", 'r') as f0:
with open("input.txt", 'r') as f0:
    text= f0.read()

In [3]:
class alm_map():
    def __init__(self, text_map):
        # text_map contains the text with lines providing map
        self.params_list=[]
        lines_map = text_map.split('\n')
        #print(lines_map)
        for l0 in lines_map:
            values_str = l0.split(' ')
            if len(values_str)!=3:
                #print(f"Error with this line {l0}, does not have 3 values")
                continue
            dest_orig = int(values_str[0])
            source_orig = int(values_str[1])
            length = int(values_str[2])
            self.params_list.append((dest_orig, source_orig, length))
    
    def source_to_dest(self,i):
        ret=i # default
        # go though list and finds the map
        for d0, s0, l0 in self.params_list:
            if i>=s0 and i<s0+l0:
                ret=d0+(i-s0)
                break
        return ret
    
    def npsource_to_npdest(self, npsource):
        # numpy 1dim array of booleans where mapped regions are masked with one

        npsource1 = npsource.copy()
        npdest=npsource.copy()

        #Check against the mapping parameters
        for d0, s0, l0 in self.params_list:
            s1=s0+l0
            d1= d0+l0
            print(f"mapping {s0}:{s1} to {d0}:{d1}, l0:{l0} to be applied")
            
            #check dimensions of destination
            max_dim= max(s1,d1)
            if npdest.shape[0]<max_dim:
                #print(f"Increasing array size to {max_dim}")
                npdest=np.resize(npdest, max_dim)
                npsource1=np.resize(npsource1,max_dim)
                #print(f"After resizing, array is {npdest}")

            nptemp = npsource1[s0:s1] #use source
            #npdest[s0:s1]=False
            npdest[d0:d1]=nptemp

        return npdest

Get starting seeds

In [4]:
seeds_inp = re.findall("seeds:(.*)", text)
seeds_inp

[' 202517468 131640971 1553776977 241828580 1435322022 100369067 2019100043 153706556 460203450 84630899 3766866638 114261107 1809826083 153144153 2797169753 177517156 2494032210 235157184 856311572 542740109']

convert to seed ranges

In [5]:
seeds_ranges=[]
s_list = seeds_inp[0].strip().split()
for i in range(0,len(s_list),2):
    s0=int(s_list[i])
    s1=int(s_list[i+1])+s0
    seeds_ranges.append((s0,s1))
seeds_ranges

[(202517468, 334158439),
 (1553776977, 1795605557),
 (1435322022, 1535691089),
 (2019100043, 2172806599),
 (460203450, 544834349),
 (3766866638, 3881127745),
 (1809826083, 1962970236),
 (2797169753, 2974686909),
 (2494032210, 2729189394),
 (856311572, 1399051681)]

In [6]:
max_dim= np.max(np.array(seeds_ranges))
max_dim

3881127745

In [7]:
seeds_np =np.zeros(max_dim, dtype=bool)
for s0,s1 in seeds_ranges:
    seeds_np[s0:s1]=True
seeds_np

array([False, False, False, ...,  True,  True,  True])

In [8]:
def get_ranges(npbools):
    #tr0 = np.where(npbools)
    #print(tr0)

    tr1 = np.where(npbools[:-1]!=npbools[1:])[0]
    tr1+=1
    #print(tr1)

    ranges=[]
    if npbools[0]: r0=0  #check status of first element
    
    for i in tr1:
        if i==len(npbools):
            break
        if npbools[i]:
            r0=i
        else:
            ranges.append([r0,i])
    if npbools[-1]:
        ranges.append([r0,len(npbools)])
    return ranges

In [9]:
get_ranges(seeds_np)

[[202517468, 334158439],
 [460203450, 544834349],
 [856311572, 1399051681],
 [1435322022, 1535691089],
 [1553776977, 1795605557],
 [1809826083, 1962970236],
 [2019100043, 2172806599],
 [2494032210, 2729189394],
 [2797169753, 2974686909],
 [3766866638, 3881127745]]

In [10]:
seeds_ranges

[(202517468, 334158439),
 (1553776977, 1795605557),
 (1435322022, 1535691089),
 (2019100043, 2172806599),
 (460203450, 544834349),
 (3766866638, 3881127745),
 (1809826083, 1962970236),
 (2797169753, 2974686909),
 (2494032210, 2729189394),
 (856311572, 1399051681)]

setup the pipeline according to the almanac

In [11]:
alm_pipeline_cls=[]

re0list = [
    "seed-to-soil map:\n((.|\n)*)\nsoil-to-fertilizer map",
    "soil-to-fertilizer map:\n((.|\n)*)\nfertilizer-to-water map:",
    "fertilizer-to-water map:\n((.|\n)*)\nwater-to-light map",
    "water-to-light map:\n((.|\n)*)\nlight-to-temperature map",
    "light-to-temperature map:\n((.|\n)*)\ntemperature-to-humidity map:",
    "temperature-to-humidity map:\n((.|\n)*)\nhumidity-to-location map:",
    "humidity-to-location map:\n((.|\n)*)\n"
]

for re0 in re0list:
    t0= re.findall(re0, text)
    t0_map = alm_map(t0[0][0])
    alm_pipeline_cls.append(t0_map)

Test pipeline with first input

In [12]:
soil_np = alm_pipeline_cls[0].npsource_to_npdest(seeds_np)
soil_np

mapping 644938450:804624157 to 1393363309:1553049016, l0:159685707 to be applied
mapping 1844060172:1863372374 to 2025282601:2044594803, l0:19312202 to be applied
mapping 1026919253:1059790345 to 1233103806:1265974898, l0:32871092 to be applied
mapping 1933428941:2019959932 to 1086566452:1173097443, l0:86530991 to be applied
mapping 0:21589659 to 1265974898:1287564557, l0:21589659 to be applied
mapping 1636167265:1671909450 to 1357621124:1393363309, l0:35742185 to be applied
mapping 2665606060:2746727202 to 2343571960:2424693102, l0:81121142 to be applied
mapping 809179011:1011676203 to 1585337376:1787834568, l0:202497192 to be applied
mapping 3039622538:3094154389 to 3151050390:3205582241, l0:54531851 to be applied
mapping 804624157:809179011 to 2059837853:2064392707, l0:4554854 to be applied
mapping 124717914:183998060 to 169037772:228317918, l0:59280146 to be applied
mapping 183998060:432113003 to 228317918:476432861, l0:248114943 to be applied
mapping 2343571960:2395245583 to 26465

MemoryError: Unable to allocate 7.23 GiB for an array with shape (7762255490,) and data type bool

Does not work. Requires 7Gb!!

In [ ]:
get_ranges(soil_np)

[[57, 70], [81, 95]]

Try the whole pipeline

In [ ]:
s0 = seeds_np

for i,p0 in  enumerate(alm_pipeline_cls):
    print(i)
    s1=p0.npsource_to_npdest(s0)
    #print("min index:",np.nonzero(s1==True)[0][0])
    #print(get_ranges(s1))
    s0=s1

#print(s1)

0
mapping 98:100 to 50:52, l0:2 to be applied
mapping 50:98 to 52:100, l0:48 to be applied
[[57, 70], [81, 95]]
1
mapping 15:52 to 0:37, l0:37 to be applied
mapping 52:54 to 37:39, l0:2 to be applied
mapping 0:15 to 39:54, l0:15 to be applied
[[57, 70], [81, 95]]
2
mapping 53:61 to 49:57, l0:8 to be applied
mapping 11:53 to 0:42, l0:42 to be applied
mapping 0:7 to 42:49, l0:7 to be applied
mapping 7:11 to 57:61, l0:4 to be applied
[[53, 57], [61, 70], [81, 95]]
3
mapping 18:25 to 88:95, l0:7 to be applied
mapping 25:95 to 18:88, l0:70 to be applied
[[46, 50], [54, 63], [74, 88]]
4
mapping 77:100 to 45:68, l0:23 to be applied
mapping 45:64 to 81:100, l0:19 to be applied
mapping 64:77 to 68:81, l0:13 to be applied
[[45, 56], [78, 81], [82, 86], [90, 99]]
5
mapping 69:70 to 0:1, l0:1 to be applied
mapping 0:69 to 1:70, l0:69 to be applied
[[46, 57], [78, 81], [82, 86], [90, 99]]
6
mapping 56:93 to 60:97, l0:37 to be applied
mapping 93:97 to 56:60, l0:4 to be applied
[[46, 61], [82, 85], [

Something is wrong going from 3 to 4, light-to-temp. I should give someting like [(78, 3), (45, 11), (90, 9), (82, 4)] (start, length)

It's ok now. I think

In [ ]:
get_ranges(s1)

[[46, 61], [82, 85], [86, 90], [94, 99]]

In [ ]:
np.min(np.array(get_ranges(s1)))

46